In [1]:
import os
import numpy as np
import math  
import matplotlib
import matplotlib.pyplot as plt
# from sklearn import linear_model
# from sklearn.linear_model import ElasticNet
# from sklearn.datasets import make_regression
# from sklearn.metrics import r2_score
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import Ridge
# from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
#                                               ExpSineSquared, DotProduct,
#                                               ConstantKernel)
import time
from sklearn.metrics import mean_squared_error
import pickle  
import datetime 
import json

In [2]:
epsilon = 1
sigma = 1
rcut = 2.5
blx = 1.1832159566199232e+02
bly = 1.1832159566199232e+02
atoms = 9800

In [5]:
filename = 'data/testFxS1Sim60T1_2R_0_7atomDataCombined.npy'
# data = np.load('c70TrainatomDataCombined.npy')
data = np.load(filename)
print('Data shape ', data.shape)

Data shape  (340323, 41)


In [8]:

# # shuffle data
N = data.shape[0]

# ids = np.arange(N)
# np.random.seed(0)
# np.random.shuffle(ids)
# data = unshuffled_data[ids]

class ObjectView(object):
    def __init__(self, d): 
        self.__dict__ = d
# data object
lamps_data = ObjectView({})

lamps_data.atom_id = data[:,0]
lamps_data.x = data[:,5]
lamps_data.y = data[:,6]
lamps_data.z = data[:,7]
lamps_data.fx = data[:,23]
lamps_data.fy = data[:,24]
lamps_data.vx = data[:,20]
lamps_data.vy = data[:,21]
lamps_data.timestep = data[:,26]
lamps_data.force1Px = data[:,27]
lamps_data.force1Py = data[:,28]
lamps_data.force1Pz = data[:,29]
lamps_data.forcexleftPx = data[:,30]
lamps_data.forcexleftPy = data[:,31]
lamps_data.forcexleftPz = data[:,32]
lamps_data.temp = data[:,33]
lamps_data.KE = data[:,34]
lamps_data.PE = data[:,35]
lamps_data.TE = data[:,36]
lamps_data.Pressure = data[:,37]
lamps_data.VxA = data[:,38]
lamps_data.VyA = data[:,39]
lamps_data.VzA = data[:,40]
lamps_data.length = N

lamps_data.x.shape
lamps_data.length

340323

In [9]:
# Assemble Xtrain and Ytrain
# Xtrain is the input

def get_Xdata_atomic(lamps_data, test_fraction=0.1):
    """ The atomic data is simply position and velocity"""
    # we're only doing 2D data
    X = np.vstack([lamps_data.x, lamps_data.y, lamps_data.vx, lamps_data.vy])
    Ntest = int(lamps_data.length * 0.1)
    Xtest = X[:, :Ntest].T
    Xtrain = X[:, Ntest:].T
    return Xtrain, Xtest

def get_Ydata(lamps_data, test_fraction=0.1):
    """ The output data is the X and Y forces"""
    # we're only doing 2D data
    Y = np.vstack([lamps_data.forcexleftPx, lamps_data.forcexleftPy])
    Ntest = int(lamps_data.length * 0.1)
    Ytest = Y[:, :Ntest].T
    Ytrain = Y[:, Ntest:].T
    return Ytrain, Ytest


In [10]:
test_fraction = 0.1
Xtrain, Xtest = get_Xdata_atomic(lamps_data, test_fraction=test_fraction)
Ytrain, Ytest = get_Ydata(lamps_data, test_fraction=test_fraction)

print(Xtrain.shape)
print(Ytrain.shape)
print(Xtest.shape)
print(Ytest.shape)

(306291, 4)
(306291, 2)
(34032, 4)
(34032, 2)


In [11]:
regressor_options = { 'hidden_layer_sizes':(40,40,40,40),
                      'random_state':0,  # Random seed 
                    }

regr = MLPRegressor(**regressor_options)

regr.fit(Xtrain, Ytrain)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(40, 40, 40, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=0, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [ ]:
# score model using r^2 metric:
score = regr.score(Xtest, Ytest)

In [ ]:
# save model
info = { 'data_filename': filename,
         'model_type': 'sklearn.neural_network.MLPRegressor',
         'score': score,
         'regressor_options': regressor_options,
        }
print(json.dumps(info, indent=1))
mlpfile = 'trained_multilayer_perceptron_' + timestamp + '.pkl'
model_folder = 'trained_models/'
def save_model(model_file, model, info):
    pickle.dump((model, info), open(model_folder + model_file, 'wb'))
    print('Model saved to %s' % model_file)
    
def load_model(model_file, model, info):
    print('Loading model from %s' % model_file)
    model, info = pickle.load(open(model_folder + model_file, 'rb'))
    print(json.dumps(info, indent=1))
    return model, info

